# Nbextension Introduction
- community-contributed and maintained plug-ins to the Jupyter notebook


**Installation:**

```bash
conda install -c conda-forge jupyter_contrib_nbextensions
conda install -c conda-forge jupyter_nbextensions_configurator
```


# bqplot: Complex Interactive Visualizations

- https://github.com/bloomberg/bqplot


bqplot is a jupyter interactive widget library bringing d3.js visualization to the Jupyter notebook.

bqplot provides both
-	high-level plotting procedures with relevant defaults for common chart types,
-	lower-level descriptions of data visualizations meant for complex interactive visualization dashboards and applications involving mouse interactions and user-provided Python callbacks.

**Installation:**

```bash
conda install -c conda-forge bqplot
```

In [1]:
from __future__ import print_function
from IPython.display import display
from ipywidgets import *
from traitlets import *

import numpy as np
import pandas as pd

In [2]:
from bqplot import DateScale, LinearScale, Axis, Lines, Figure
from bqplot.interacts import (FastIntervalSelector, IndexSelector)

from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML

In [2]:
from bqplot import DateScale, LinearScale, Axis, Lines, Scatter, Bars, Hist, Figure
from bqplot.interacts import (
    FastIntervalSelector, IndexSelector, BrushIntervalSelector,
    BrushSelector, MultiSelector, LassoSelector, PanZoom, HandDraw
)
from traitlets import link

from ipywidgets import ToggleButtons, VBox, HTML


In [53]:
import yfinance as yf
df_aapl=yf.download('AAPL',start='2020-02-01',end='2021-04-15',progress=False,auto_adjust=True)
df_dis=yf.download('DIS',start='2020-02-01',end='2021-04-15',progress=False,auto_adjust=True)

In [54]:
df_price=pd.concat([df_aapl['Close'],df_dis['Close']],axis=1)
df_price.columns=['AAPL','DIS']
df_price.head()

,AAPL,DIS
Date,,
2020-02-03,76.390160,141.320007
2020-02-04,78.912079,144.729996
2020-02-05,79.555550,141.369995
2020-02-06,80.486107,142.699997
2020-02-07,79.392090,141.020004


In [55]:
df_price.to_csv('Stock.csv',index=True)

In [3]:
price_data=pd.read_csv('Stock.csv')

symbol='AAPL'
dates_actual = pd.to_datetime(price_data['Date']).dt.normalize().values
aaplprices = price_data[symbol].values
disprices=price_data['DIS'].values
#price_data2.columns=['AAPL','DIS']

## Interval Selector

In [24]:
from bqplot import pyplot as plt

In [4]:
dt_x_fast = DateScale()
lin_y = LinearScale()

x_ax = Axis(label='Date', scale=dt_x_fast)
x_ay = Axis(label='Price', scale=lin_y, orientation='vertical')
AAPL = Lines(x=dates_actual, y=aaplprices, scales={'x': dt_x_fast, 'y': lin_y}, colors=['orange'],labels=['AAPL'],display_legend=True)
DIS = Lines(x=dates_actual, y=disprices, scales={'x': dt_x_fast, 'y': lin_y}, colors=['blue'],labels=['DIS'],display_legend=True)

In [5]:
intsel_fast = FastIntervalSelector(scale=dt_x_fast, marks=[AAPL])

In [6]:
def fast_interval_change_callback(change):
    db_fast.value = 'The selected period is ' + str(change.new)
intsel_fast.observe(fast_interval_change_callback, names=['selected'])

In [7]:
db_fast = HTML()
db_fast.value = 'The selected period is ' + str(intsel_fast.selected)

fig_fast_intsel = Figure(marks=[AAPL,DIS], axes=[x_ax, x_ay], title='Stock Prices',legend_location='bottom',
                         interaction=intsel_fast) #This is where we assign the interaction to this particular Figure

VBox([db_fast, fig_fast_intsel])

## Index Selector

In [8]:
db_index = HTML(value='[]')
index_sel = IndexSelector(scale=dt_x_fast, marks=[AAPL, DIS])

In [9]:
def index_change_callback(change):
    db_index.value = 'The selected date is ' + str(change.new)

In [10]:
index_sel.observe(index_change_callback, names=['selected'])


In [11]:
fig_index_sel = Figure(marks=[AAPL, DIS], axes=[x_ax, x_ay], title='Index Selector Example',legend_location='top-left',
                       interaction=index_sel)
VBox([db_index, fig_index_sel])


In [16]:
from datetime import datetime as py_dtime

dt_x_index = DateScale(min=np.datetime64(py_dtime(2020, 2, 1)))
lin_y2 = LinearScale()

lc2_index = Lines(x=dates_actual, y=aaplprices,
            scales={'x': dt_x_index, 'y': lin_y2})

x_ax1 = Axis(label='Date', scale=dt_x_index)
x_ay2 = Axis(label=(' Price'), scale=lin_y2, orientation='vertical')

In [17]:
intsel_date = FastIntervalSelector(scale=dt_x_index, marks=[lc2_index])

In [20]:
db_date = HTML()
db_date.value = str(intsel_date.selected)


In [21]:
def date_interval_change_callback(change):
    db_date.value = str(change.new)

In [22]:
lc2_index.observe(date_interval_change_callback, names=['selected'])

fig_date_mark = Figure(marks=[lc2_index], axes=[x_ax1, x_ay2],
                       title='Fast Interval Selector Selected Indices Example', interaction=intsel_date)

VBox([db_date, fig_date_mark])